# License Plate Recognition using TAO LPRNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## Sample prediction of LPRNet
<img align="center" src="./LPRNet.png" width="480">

## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained baseline18 LPRNet model and train it on the OpenALPR benchmark dataset
* Run Inference on the trained model
* Export the trained model to a .onnx file for deployment to DeepStream
* Run inference on the exported .onnx model to verify deployment using TensorRT

At the end of this notebook, you will have generated a trained and optimized `license plate recognition` model, 
which you may deploy via [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of LPRNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)
8. [Verify the deployed model](#head-8)

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/lprnet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*


In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variable with your key.")
%env KEY=nvidia_tlt
%env GPU_INDEX=0
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/lprnet
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data/openalpr

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/lprnet
%env LOCAL_PROJECT_DIR=/data/tlt-experiments
os.environ["LOCAL_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "openalpr")
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "lprnet")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=/data/tlt-experiments/lprnet
# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/lprnet/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

In [ ]:
# Installing the dependencies
import os
os.environ["PROJECT_DIR"] = os.getcwd()
!pip3 install Cython==0.29.36
!pip3 install -r $PROJECT_DIR/../deps/requirements-pip.txt

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.


In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the OpenALPR benchmark dataset for the tutorial. The following script will download the dataset automatically and convert it to the format used by TAO. 

In [ ]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

In [ ]:
!bash download_and_prepare_data.sh $LOCAL_DATA_DIR

In [ ]:
# verify
!ls -l $LOCAL_DATA_DIR/
!ls -l $LOCAL_DATA_DIR/train
!ls -l $LOCAL_DATA_DIR/train/image
!ls -l $LOCAL_DATA_DIR/train/label


## 2.1 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/lprnet:*

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/lprnet:trainable_v1.0 --dest $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0

## 2. Provide training specification <a class="anchor" id="head-2"></a>

* Note the spec $SPEC_DIR/default_sepc.txt is for training on US license plates:
    * the max license plate length is 8;
        * You can change `max_label_length` in `lpr_config` to satisfy your own dataset.
    * the characters of US license plates are: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, C, D, E, F, G, H, I, J, K, L, M, N, P, Q, R, S, T, U, V, W, X, Y, Z 
        * You can change `characters_list_file` in `dataset_config` to set your own characters.
        * `characters_list_file` should contain all the characters in dataset. And one character takes one line. 

In [ ]:
!cat $LOCAL_SPECS_DIR/tutorial_spec.txt

In [ ]:
!cat $LOCAL_SPECS_DIR/us_lp_characters.txt

## 3. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

In [ ]:
print("For multi-GPU, change --gpus based on your machine.")
!tao model lprnet train --gpus=1 --gpu_index=$GPU_INDEX \
                  -e $SPECS_DIR/tutorial_spec.txt \
                  -k $KEY \
                  -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                  -m $USER_EXPERIMENT_DIR/pretrained_lprnet_baseline18/lprnet_vtrainable_v1.0/us_lprnet_baseline18_trainable.tlt

In [ ]:
print("To resume training from a checkpoint, set the -m option to be the .tlt you want to resume from and --initial_epochs to be the epoch index of the resumed checkpoint")
# !tao model lprnet train --gpu_index=$GPU_INDEX \
#                   -e $SPECS_DIR/tutorial_spec.txt \
#                   -k $KEY \
#                   -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
#                   -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-001.hdf5
#                   --initial_epoch 2

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights/

## 4. Evaluate trained models <a class="anchor" id="head-4"></a>

In [ ]:
!tao model lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                     -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-024.hdf5

## 5. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the lprnet inference tool to generate inferences on the trained models and print the results. 

In [ ]:
# Running inference for detection on n images
!tao model lprnet inference --gpu_index=$GPU_INDEX -i $DATA_DOWNLOAD_DIR/val/image \
                      -e $SPECS_DIR/tutorial_spec.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-024.hdf5

## 6. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
# Generate .onnx file using tao container
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 
!tao model lprnet export --gpu_index=$GPU_INDEX -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/lprnet_epoch-024.hdf5 \
                   -e $SPECS_DIR/tutorial_spec.txt \
                   --target_opset 12 \
                   -o $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.onnx

Using the `tao deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference. 

The `tao deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao deploy` command which will instantiate a deploy container, with the exported `.onnx` file on your target device. The `tao deploy` container only works for x86, with discrete NVIDIA GPU's. 

If you choose to integrate your model into deepstream directly, please refer to [deepstream dev guide](https://docs.nvidia.com/tao/tao-toolkit/text/ds_tao/deepstream_tao_integration.html) for more details.

In [ ]:
# Convert to TensorRT engine(FP32).
!tao deploy lprnet gen_trt_engine --gpu_index=$GPU_INDEX -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.onnx \
                                  --data_type fp32 \
                                  --engine_file $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.fp32.engine \
                                  --results_dir $USER_EXPERIMENT_DIR/export

In [ ]:
# Verify the tensorrt engine accuracy on the validation dataset through tao deploy
!tao deploy lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                             -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.fp32.engine \
                             --results_dir $USER_EXPERIMENT_DIR/evaluate

In [ ]:
# Check if onnx model is correctly saved.
!ls -l $LOCAL_EXPERIMENT_DIR/export

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $LOCAL_EXPERIMENT_DIR/export

In [ ]:
# Convert to TensorRT engine(FP16).
!tao deploy lprnet gen_trt_engine --gpu_index=$GPU_INDEX -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.onnx \
                                  --data_type fp16 \
                                  --engine_file $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.fp16.engine \
                                  --min_batch_size 1 \
                                  --opt_batch_size 4 \
                                  --max_batch_size 16 \
                                  --results_dir $USER_EXPERIMENT_DIR/export

In [ ]:
print('Exported engine:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/

## 7. Verify the deployed model <a class="anchor" id="head-7"></a>

Verify the converted engine.

In [ ]:
# Running evaluation on validation dataset through tao deploy
!tao deploy lprnet evaluate --gpu_index=$GPU_INDEX -e $SPECS_DIR/tutorial_spec.txt \
                            -m $USER_EXPERIMENT_DIR/export/lprnet_epoch-024.fp16.engine \
                            --results_dir $USER_EXPERIMENT_DIR/evaluate